# Hyperparameter Tuning: Learning Rate and Metrics

Welcome to this first exploration of hyperparameter tuning using the [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset!  This dataset consists on 60,000 32x32 color images in 10 classes (6,000 images per class). Here are the classes in the dataset, as well as 10 random images from each: 

![](./nb_image/cifar10.png)

In this notebook, you'll focus specifically on the learning rate, an essential hyperparameter that dictates the pace at which a model learns during training. You'll work with a simple convolutional neural network (CNN) and observe how changes in hyperparameters affect the model's outcomes.

This lab will cover the following:

* Examining the effects of different **learning rates** on model performance.

* Introducing and using additional metrics like **precision, recall, and F1 score** for a more complete evaluation.

* Exploring the effect of **batch size** on different metrics using an imbalanced dataset in an optional section.

In [4]:
import pandas as pd 
import torch
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim
import torchmetrics

import helper

helper.set_seed(42)

device = torch.device("mps" if torch.mps.is_available() else "cpu")
print(device)

mps


## Learning Rate Optimization on CIFAR-10

You will now explore how the learning rate affects the performance of a simple convolutional neural network (CNN) on the CIFAR-10 dataset.
For that you will train a simple CNN model with different learning rates and observe the validation accuracy to understand the sensitivity of the model to this hyperparameter.

The code below sets up the necessary functions for training the model and evaluating its performance:

- `SimpleCNN`: defines a small convolutional neural network architecture.

- `evaluate_accuracy`: computes the accuracy on a validation dataset.

These functions encapsulate the key elements required for running the hyperparameter optimization experiments.

In [5]:
class SimpleCNN(nn.Module):
    """A simple Convolutional Neural Network (CNN) architecture.

    This class defines a two-layer CNN with max pooling, dropout, and
    fully connected layers, suitable for basic image classification tasks.
    """

    def __init__(self):
        """Initialize the layers of the neural network."""

        #Initialize the parent nn.Module class
        super().__init__()

        #First convolution layer (3 input channels, 16 output channels , 3X3 kernel)
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        #Second con layer
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)

        #Max pooling layer with 2X2 windows and stride of 2
        self.pool = nn.MaxPool2d(2,2)

        #First fully connected liner layer
        self.fc1 = nn.Linear(32 * 8 * 8, 64)

        #Second fully connected lauer serving as a output layer
        self.fc2 = nn.Linear(64, 10)

        #Dropout layer for regularization
        self.droput = nn.Dropout(p=0.4)

    def forward(self, x):
        """Defines the forward pass of the network.

        Args:
            x (torch.Tensor): The input tensor of shape (batch_size, 3, height, width).

        Returns:
            torch.Tensor: The output logits from the network.
        """

        #Apply first convolution, ReLU activation and max pooling
        x = self.pool(F.relu(self.conv1(x)))

        #Apply second convolution, ReLU activation, and max poling
        x = self.pool(F.relu(self.conv2(x)))

        #Flatten the feature maps for the fully connected layers
        x = x.view(-1, 32*8*8)

        #Apply first fully connected layer with ReLU activation
        x = F.relu(self.fc1(x))

        #Apply dropout for regulatization
        x = self.droput(x)

        #Apply the final outputlayer
        x = self.fc2(x)

        return x
